In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv('data.csv', index_col = 0)

In [3]:
data = data.assign(Ret1M  = data.Close.groupby(data.Tick).pct_change(1))
data = data.assign(Ret3M  = data.Close.groupby(data.Tick).pct_change(3))
data = data.assign(Ret6M  = data.Close.groupby(data.Tick).pct_change(6))
data = data.assign(Ret12M = data.Close.groupby(data.Tick).pct_change(12))

In [4]:
data = data.assign(MOM3M  = data.Close.groupby(data.Tick).shift(1)/data.Close.groupby(data.Tick).shift(4)  - 1)
data = data.assign(MOM6M  = data.Close.groupby(data.Tick).shift(1)/data.Close.groupby(data.Tick).shift(6)  - 1)
data = data.assign(MOM12M = data.Close.groupby(data.Tick).shift(1)/data.Close.groupby(data.Tick).shift(12) - 1)

In [5]:
# Changing "Date" to datetime format
data['Date'] = pd.to_datetime(data['Date'])

In [6]:
data.dropna(axis=0, how='any',inplace=True) # Drop first 12 rows since we dont' have some factors (e.g. ret12) # Also, we might now need data from 1999 to predict current movements

In [7]:
#Sets index of dataframe to the stock's tick for sorting purposes
data.set_index("Tick",inplace=True)

### Standardising the data and calculating Quintiles

In [8]:
%%capture
numeric_columns = list(data.columns.values[data.dtypes.values == float]) # Keeeping only numeric columns

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1)) #Defyning the scalar object

def get_cross_section(data,date): # Used
    return data[data.Date == date]

standardized_data = pd.DataFrame(columns = list(data.columns.values).append("Quintile"))

for date in set(data.Date.values):
    CS_numeric_date = get_cross_section(data,date)[numeric_columns] # Extracting numeric columns from the cross section
    transformed = scaler.fit_transform(CS_numeric_date) # cross-sectional standardization
    scaled_features_df = pd.DataFrame(transformed, index=CS_numeric_date.index, columns=CS_numeric_date.columns) # transforming into dataframe
    scaled_features_df.insert(0, "Date", date) # Adding back the corresponding date
    
    
    ################ Calculating quintiles ##################
    q_20 = scaled_features_df["Ret1M"].quantile(q=0.2, interpolation='linear')
    q_40 = scaled_features_df["Ret1M"].quantile(q=0.4, interpolation='linear')
    q_60 = scaled_features_df["Ret1M"].quantile(q=0.6, interpolation='linear')
    q_80 = scaled_features_df["Ret1M"].quantile(q=0.8, interpolation='linear')
    
    # Encoding: 4 is in 1st quantile, 3 if in second quantile ...
    idx1 = np.where((scaled_features_df['Ret1M']<= q_20))
    idx2 = np.where((scaled_features_df['Ret1M']> q_20) & (scaled_features_df['Ret1M']<= q_40))
    idx3 = np.where((scaled_features_df['Ret1M']> q_40) & (scaled_features_df['Ret1M']<= q_60))
    idx4 = np.where((scaled_features_df['Ret1M']> q_60) & (scaled_features_df['Ret1M']<= q_80))
    idx5 = np.where((scaled_features_df['Ret1M']> q_80))
    
    scaled_features_df["Quintile"] = 999 # Initializing value
    scaled_features_df["Quintile"].iloc[idx1] = 0
    scaled_features_df["Quintile"].iloc[idx2] = 1
    scaled_features_df["Quintile"].iloc[idx3] = 2
    scaled_features_df["Quintile"].iloc[idx4] = 3
    scaled_features_df["Quintile"].iloc[idx5] = 4
    
    
    standardized_data = pd.concat([standardized_data, scaled_features_df]) # Adding the data from this date to the whole standardized database
    
    
standardized_data.sort_values(by=['Date'],inplace = True)

### Defining the Target Variable (Shifted Quintiles)

We want to predict the stock quintile at time t_1 using the data at time t_0. Therefore we now add the stock quintile for the following month to each timestep, so that we can have the target variable for our model.

In [9]:
%%capture
standardized_data_with_target = pd.DataFrame(columns = list(standardized_data.columns.values).append("Target_Shifted_Quintile_1"))

for tick in set(standardized_data.index.values):
    TS_tick = standardized_data[standardized_data.index == tick]
    
    
    TS_tick["Target_Shifted_Quintile_1"] = TS_tick[["Quintile"]].shift(-1)
    TS_tick["Target_Shifted_Quintile_1"][-1] = 999 # Before converting to int we have to get rid of the nan (last value of the shifted)
    TS_tick["Target_Shifted_Quintile_1"] = TS_tick["Target_Shifted_Quintile_1"].astype(int)
    
    standardized_data_with_target = pd.concat([standardized_data_with_target, TS_tick]) # Adding the data from this date to the whole standardized database
    
    
standardized_data_with_target["Temp"] = standardized_data_with_target.index
standardized_data_with_target.sort_values(by=['Date', 'Temp'], inplace=True)
standardized_data_with_target.sort_values(by=['Date', 'Temp'], inplace=True)
standardized_data_with_target.drop(['Temp'],axis='columns', inplace=True)

### Drop the last timestep - We don't have the target for it

In [10]:
standardized_data_with_target.reset_index(level=0, inplace=True)

standardized_data_with_target.drop(standardized_data_with_target[standardized_data_with_target.Target_Shifted_Quintile_1 == 999].index,axis='rows',inplace = True)

standardized_data_with_target.set_index("Tick",inplace = True)

In [12]:
standardized_data_with_target[standardized_data_with_target.index == "AAPL"]

,Date,Close,Mkt_Cap,P2B,Vol_1M,Div_yield,PE_ratio,RSI_1M,D2E,Prof_growth,...,Prof_Marg,Ret1M,Ret3M,Ret6M,Ret12M,MOM3M,MOM6M,MOM12M,Quintile,Target_Shifted_Quintile_1
Tick,,,,,,,,,,,,,,,,,,,,,
AAPL,2000-02-01,0.022677,0.036030,0.072826,0.600162,0.012588,0.008101,0.520843,0.005611,0.123616,...,0.381461,0.424180,0.575431,0.840107,0.745640,1.000000,1.000000,0.986130,1,4
AAPL,2000-03-01,0.032247,0.048609,0.016235,0.524846,0.009376,0.010422,0.849844,0.000829,0.123616,...,0.381461,0.586206,0.475049,0.583603,0.764580,0.575431,0.600636,0.839929,4,1
AAPL,2000-04-03,0.030162,0.040499,0.016246,0.524739,0.015398,0.007507,0.614990,0.000868,0.078646,...,0.742988,0.358701,0.511432,0.787714,0.986323,0.475049,0.669820,1.000000,1,0
AAPL,2000-05-01,0.025905,0.038179,0.014986,0.607509,0.016104,0.007214,0.529778,0.000868,0.078646,...,0.742988,0.304270,0.420294,0.572176,0.601535,0.511432,0.688308,0.810863,0,0
AAPL,2000-06-01,0.017904,0.027837,0.009291,0.537481,0.013300,0.004799,0.111120,0.000868,0.078646,...,0.742988,0.000000,0.031114,0.242497,0.534550,0.420294,0.471531,0.757179,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AAPL,2017-12-01,0.430192,1.000000,0.004364,0.194933,0.150066,0.011055,0.539397,0.003472,0.124014,...,0.930455,0.377336,0.406470,0.457753,0.734036,0.514707,0.497928,0.813081,2,2
AAPL,2018-01-02,0.413368,1.000000,0.004102,0.136336,0.149001,0.036222,0.604077,0.003513,0.278233,...,0.610201,0.362310,0.599364,0.495724,0.709387,0.406470,0.588280,0.744957,2,1
AAPL,2018-02-06,0.399474,1.000000,0.003884,0.294773,0.157526,0.037852,0.402585,0.003513,0.278233,...,0.610201,0.545467,0.312919,0.520593,0.517973,0.599364,0.476597,0.601977,1,4
